In [1]:
import tensorflow as tf

In [2]:
from songs import *

import copy
def addAllTransposedVersions(Songs, song):
    while song.transpose(1):
        pass

    Songs.append(copy.deepcopy(song))
    while song.transpose(-1):
        Songs.append(copy.deepcopy(song))
        
Songs = []
for i in range(1, 35):
    addAllTransposedVersions(Songs, Song('basic midi/track (' + str(i) + ').mid'))
    
all_songs = np.array([song.notes for song in Songs])

ERROR! out of range!
ERROR! out of range!
ERROR! out of range!


In [5]:
def generate():
    X = np.random.randint(0, 2, (128, 128+64, 13))
    X[:, 128:, :] = 0
    Y = np.zeros((128, 128+64, 13))
    Y[:, 64:, :] = X[:, :128, :]
    return X, Y

In [28]:
tf.reset_default_graph()

# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 13
timesteps = 128+64 # timesteps
num_hidden = 50 # hidden layer num of features
history_size = 80

# tf Graph input
X = tf.placeholder("float", [batch_size, timesteps, num_input])
Y = tf.placeholder("float", [batch_size, timesteps, num_input])

loss_op = tf.constant(0.0)

read_lstm = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0, reuse=None)
read_lstm_state = tf.zeros([batch_size, num_hidden]), tf.zeros([batch_size, num_hidden])

read_w = tf.Variable(tf.random_normal([num_hidden, history_size]))
read_b = tf.Variable(tf.random_normal([history_size]))

history = tf.zeros([batch_size, num_input, history_size])

delete_lstm = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0, reuse=True)
delete_lstm_state = tf.zeros([batch_size, num_hidden]), tf.zeros([batch_size, num_hidden])

delete_w = tf.Variable(tf.random_normal([num_hidden, history_size]))
delete_b = tf.Variable(tf.random_normal([history_size]))

write_lstm = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0, reuse=True)
write_lstm_state = tf.zeros([batch_size, num_hidden]), tf.zeros([batch_size, num_hidden])

write_w = tf.Variable(tf.random_normal([num_hidden, history_size]))
write_b = tf.Variable(tf.random_normal([history_size]))

_X = tf.unstack(X, timesteps, 1)
_Y = tf.unstack(Y, timesteps, 1)
for t, inp, truth in zip(np.arange(timesteps), _X, _Y):
    read_lstm_output, read_lstm_state = read_lstm(inp, read_lstm_state)
    
    read = tf.sigmoid(tf.matmul(read_lstm_output, read_w) + read_b)
    memory_retrieve = tf.matmul(history, tf.expand_dims(read, 2))
    if t >= 64:
        loss_op += tf.reduce_mean(tf.squared_difference(tf.squeeze(memory_retrieve, 2), truth))
        
    delete_lstm_output, delete_lstm_state = delete_lstm(inp, delete_lstm_state)
    delete = tf.sigmoid(tf.matmul(delete_lstm_output, delete_w) + delete_b)
    history = history * tf.expand_dims(delete, 1)
    
    write_lstm_output, write_lstm_state = write_lstm(inp, write_lstm_state)
    write = tf.sigmoid(tf.matmul(write_lstm_output, write_w) + write_b)
    history = history + tf.tile(tf.expand_dims(inp, 2), [1, 1, history_size]) * tf.expand_dims(write, 1)

# Define loss and optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = tf.contrib.layers.optimize_loss(
    loss_op, tf.contrib.framework.get_global_step(), optimizer=optimizer,
    learning_rate=learning_rate)

init = tf.global_variables_initializer()

In [29]:
# Start training
sess = tf.Session()
# Run the initializer
sess.run(init)

for step in range(0, training_steps):
    batch_x, batch_y = generate()
    batch_x = batch_x.reshape((batch_size, timesteps, num_input))
    batch_y = batch_y.reshape((batch_size, timesteps, num_input))
    
    # Run optimization op (backprop)
    sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
    if step % display_step == 0:
        # Calculate batch loss and accuracy
        L = sess.run(loss_op, feed_dict={X: batch_x, Y: batch_y})
        print("Step " + str(step) + ", Minibatch Loss= " + "{:.4f}".format(L))

print("Optimization Finished!")

Step 0, Minibatch Loss= 6105.5024
Step 200, Minibatch Loss= 33.6361
Step 400, Minibatch Loss= 33.0166
Step 600, Minibatch Loss= 32.7883
Step 800, Minibatch Loss= 32.6837
Step 1000, Minibatch Loss= 32.6015
Step 1200, Minibatch Loss= 32.5325
Step 1400, Minibatch Loss= 32.4914
Step 1600, Minibatch Loss= 32.4659
Step 1800, Minibatch Loss= 32.4274
Step 2000, Minibatch Loss= 32.4243
Step 2200, Minibatch Loss= 32.3921
Step 2400, Minibatch Loss= 32.3526
Step 2600, Minibatch Loss= 32.3493
Step 2800, Minibatch Loss= 32.3351
Step 3000, Minibatch Loss= 32.3062
Step 3200, Minibatch Loss= 32.3397
Step 3400, Minibatch Loss= 32.3150
Step 3600, Minibatch Loss= 32.3197
Step 3800, Minibatch Loss= 32.2563
Step 4000, Minibatch Loss= 32.2725
Step 4200, Minibatch Loss= 32.2885
Step 4400, Minibatch Loss= 32.2791
Step 4600, Minibatch Loss= 32.2352
Step 4800, Minibatch Loss= 32.2479
Step 5000, Minibatch Loss= 32.2520
Step 5200, Minibatch Loss= 32.2502
Step 5400, Minibatch Loss= 32.2343
Step 5600, Minibatch Loss

In [30]:
mr = sess.run(memory_retrieve, feed_dict={X: batch_x, Y: batch_y})

In [32]:
mr

array([[[ 0.43326914],
        [ 0.4035112 ],
        [ 0.45718867],
        [ 0.42589644],
        [ 0.42204136],
        [ 0.41942936],
        [ 0.40818581],
        [ 0.41715679],
        [ 0.45069501],
        [ 0.48232254],
        [ 0.43987894],
        [ 0.41183034],
        [ 0.51747382]],

       [[ 0.37604108],
        [ 0.41703373],
        [ 0.50206643],
        [ 0.41508847],
        [ 0.41996771],
        [ 0.399376  ],
        [ 0.49811107],
        [ 0.45637995],
        [ 0.46809542],
        [ 0.46985602],
        [ 0.45056614],
        [ 0.41279545],
        [ 0.40153477]],

       [[ 0.45060718],
        [ 0.44985259],
        [ 0.43450534],
        [ 0.44470152],
        [ 0.3959471 ],
        [ 0.412112  ],
        [ 0.43143749],
        [ 0.40608782],
        [ 0.44066274],
        [ 0.40662193],
        [ 0.47808516],
        [ 0.47203326],
        [ 0.43559146]],

       [[ 0.37947971],
        [ 0.42742413],
        [ 0.45122817],
        [ 0.36508831],
     